In [5]:
import argparse
import copy
import glob
import logging
import math
import multiprocessing
import os
import sys
from pathlib import Path
from typing import Any, Iterable, List, Mapping, Sequence, Tuple, Union

import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np

import PIL.Image as Image

import argoverse
from argoverse.utils import calibration
from argoverse.data_loading.object_label_record import json_label_dict_to_obj_record
from argoverse.data_loading.simple_track_dataloader import SimpleArgoverseTrackingDataLoader
from argoverse.map_representation.map_api import ArgoverseMap
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
from argoverse.utils.calibration import (
    CameraConfig,
    get_calibration_config,
    point_cloud_to_homogeneous,
    project_lidar_to_img_motion_compensated,
    project_lidar_to_undistorted_img,
    load_calib,
    Calibration
)
from argoverse.utils.camera_stats import (
        RING_CAMERA_LIST,
        STEREO_CAMERA_LIST,
        RING_IMG_HEIGHT,
        RING_IMG_WIDTH
)
from argoverse.utils.city_visibility_utils import clip_point_cloud_to_visible_region
from argoverse.utils.cv2_plotting_utils import draw_clipped_line_segment
from argoverse.utils.ffmpeg_utils import write_nonsequential_idx_video
from argoverse.utils.frustum_clipping import (
        generate_frustum_planes,
        cuboid_to_2d_frustum_bbox
)
from argoverse.utils.ply_loader import load_ply
from argoverse.utils.se3 import SE3
import argoverse.visualization.visualization_utils as viz_util

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger(__name__)

#: Any numeric type
Number = Union[int, float]

name_of_camera = 'ring_front_center'

In [6]:
a = [1,2,3,4]
a.sort()
a

[1, 2, 3, 4]

In [7]:
def overlaps(bbox1_xmin, bbox1_ymin, bbox1_xmax, bbox1_ymax, bbox2_xmin, bbox2_ymin, bbox2_xmax, bbox2_ymax, threshold=0.50):
#     print (bbox1_xmin, bbox1_ymin, bbox1_xmax, bbox1_ymax, bbox2_xmin, bbox2_ymin, bbox2_xmax, bbox2_ymax)
    if bbox1_xmin > bbox2_xmax or bbox2_xmin > bbox1_xmax: #gap in x
        return False
    elif bbox2_ymin > bbox2_ymax or bbox2_ymin > bbox1_ymax: #gap in y
        return False
    else: 
        area1 = (bbox1_xmax - bbox1_xmin)*(bbox1_ymax - bbox1_ymin)
        area2 = (bbox2_xmax - bbox2_xmin)*(bbox2_ymax - bbox2_ymin)
        xs = [bbox1_xmin,bbox1_xmax,bbox2_xmin,bbox2_xmax]
        ys = [bbox1_ymin,bbox1_ymax,bbox2_ymin,bbox2_ymax]
        xs.sort()
        ys.sort()
        if area1 == 0 or area2 == 0: 
            print (bbox1_xmin, bbox1_ymin, bbox1_xmax, bbox1_ymax, bbox2_xmin, bbox2_ymin, bbox2_xmax, bbox2_ymax)

        area_overlap = (xs[2] - xs[1])*(ys[2] - ys[1]) # interior points are always the overlaping ones
#         if max(area_overlap / area1 , area_overlap /area2) > threshold:
        if (area_overlap / area1) > threshold:
            return True
        else:
            return False
# 1710 572 1919 805 1919 558 1919 820
def shrink_box(bbox_xmin, bbox_ymin, bbox_xmax, bbox_ymax):
    x_avg = (bbox_xmin + bbox_xmax)/2
    y_avg = (bbox_ymin + bbox_ymax)/2
    return ((bbox_xmin + x_avg)/2, (bbox_ymin + y_avg)/2, (bbox_xmax + x_avg)/2, (bbox_ymax + y_avg)/2)
   
def ordering(uv1,xmin,ymin,xmax,ymax):
    uv1 = uv1.T
    xmin,ymin,xmax,ymax = shrink_box(xmin,ymin,xmax,ymax)
    lidar_bbox = uv1[(xmin < uv1[:,0]) & (uv1[:,0] < xmax)& (ymin < uv1[:,1]) & (uv1[:,1] < ymax)]
    if len(lidar_bbox[:,2]) != 0:
        return  np.mean(lidar_bbox[:,2])
    else:
        return 500
def pickle_format(bboxes,img):
    bboxes = [{'bbox': [xmin,xmax,ymin,ymax], 'label': label, 'id':track_id} for (xmin, ymin), (xmax, ymax),track_id,label , in bboxes]
    return {'center_camera_feed':img,'obstacles':bboxes}


def plot_img_2d_bboxes(
        labels,
        planes,
        img_bgr: np.ndarray,
        log_calib_data,
        camera_name: str,
        cam_timestamp: int,
        lidar_timestamp: int,
        data_dir: str,
        log_id: str,
        save_img_fpath: str,
        lidar_pts,
        show_depthmap = False
        
    
):
        """ """
        ############  Process lidar ###############
#         img = argoverse_data.get_image_sync(idx,camera = camera_name)
#         objects = argoverse_data.get_label_object(idx)
        
#         calib = argoverse_data.get_calibration(camera_name) # replace this with manual 
        
        cam_config = get_calibration_config(log_calib_data, camera_name)        
        calib_cam = next(
            (c for c in log_calib_data["camera_data_"] if c["key"] == f"image_raw_{camera_name}"),
            None,
        )
        if calib_cam is None:
            print (f"No Camera of name: {camera_name}")
        calib = Calibration(cam_config, calib_cam)
        
        
#         img_vis = viz_util.show_image_with_boxes(img,objects,calib)
#         display(Image.fromarray(img_vis))
        # ALMOST DEFINITELY OUT OF SYNC HERE
#         pc = argoverse_data.get_lidar(idx)
        pc = lidar_pts
        uv = calib.project_ego_to_image(pc).T
        

        idx_ = np.where(np.logical_and.reduce((uv[0, :] >= 0.0, uv[0, :] < np.shape(img_bgr)[1] - 1.0,
                                                          uv[1, :] >= 0.0, uv[1, :] < np.shape(img_bgr)[0] - 1.0,
                                                          uv[2, :] > 0)))

        # uv :: x , y , distance
        idx_ = idx_[0]

        uv1 =uv[:, idx_]

        if uv1 is None:
            raise Exception('No point image projection')

#         plt.figure(figsize=(10,30))
#         plt.imshow(img)
        
        
#         cm = plt.cm.get_cmap('jet')
#         plt.scatter(uv1[0], uv1[1], c=1 - uv1[2] / max(uv1[2]), s=1, cmap=cm)
#         plt.axis('off')
#         plt.savefig("base_"+save_img_fpath)
#         plt.clf()
#         assert 1 == 2
        
        bboxes = []
        print (len(labels))
        for label_idx, label in enumerate(labels):
                obj_rec = json_label_dict_to_obj_record(label)
                track_id = (obj_rec.track_id).replace("-","")
                track_id_short = int(track_id,16) % sys.maxsize
                if obj_rec.label_class in ["ANIMAL", "STROLLER", "OTHER_MOVER", "ON_ROAD_OBSTACLE"]:
                        continue                
                if obj_rec.occlusion == 100:
                        continue
                if obj_rec.occlusion > 0:
                        print("Occlusion {}".format(obj_rec.occlusion))
#                 print (obj_rec.label_class)
                cuboid_vertices = obj_rec.as_3d_bbox()
                points_h = point_cloud_to_homogeneous(cuboid_vertices).T

                uv, uv_cam, valid_pts_bool, camera_config = project_lidar_to_img_motion_compensated(
                        points_h,  # these are recorded at lidar_time
                        copy.deepcopy(log_calib_data),
                        camera_name,
                        cam_timestamp,
                        lidar_timestamp,
                        data_dir,
                        log_id,
                        return_K=True,
                )
                K = camera_config.intrinsic

                if valid_pts_bool.sum() == 0:
                    continue
                bbox_2d = cuboid_to_2d_frustum_bbox(uv_cam.T[:,:3], planes, K[:3,:3])
                if bbox_2d is None:
                        continue
                else:
                        x1,y1,x2,y2 = bbox_2d

                        x1 = min(x1,RING_IMG_WIDTH-1)
                        x2 = min(x2,RING_IMG_WIDTH-1)
                        y1 = min(y1,RING_IMG_HEIGHT-1)
                        y2 = min(y2,RING_IMG_HEIGHT-1)

                        x1 = max(x1, 0)
                        x2 = max(x2, 0)
                        y1 = max(y1, 0)
                        y2 = max(y2, 0)

                        xmin = min(x1, x2)
                        xmax = max(x1, x2)
                        ymin = min(y1, y2)
                        ymax = max(y1, y2)
                        start = (int(xmin), int(ymin))
                        end = (int(xmax), int(ymax))
                        if xmin != xmax and ymin != ymax: #get rid of zero area boxes
                            bboxes.append((start,end,track_id_short,obj_rec.label_class))
                        
                        
#                         cv2.rectangle(img_bgr, start, end, (0, 0, 255))

        average_depth = [ ordering(uv1, xmin,ymin,xmax,ymax) for (xmin,ymin),(xmax,ymax),_,_ in bboxes]
        new_bboxes = []
#         print (bboxes[40], bboxes[0])
#         (xmin,ymin), (xmax,ymax) = bboxes[40]
#         (x_min, y_min), (x_max,y_max) = bboxes[0]
#         print (overlaps(xmin,ymin,xmax,ymax,x_min,y_min,x_max,y_max))
        for i in range(len(bboxes)):
            (bbox1_xmin, bbox1_ymin), (bbox1_xmax, bbox1_ymax),bbox1_id,bbox1_label = bboxes[i]
            is_occluded = False
            for j  in range(len(bboxes)):
                if i != j:
                    (bbox2_xmin, bbox2_ymin), (bbox2_xmax, bbox2_ymax),_,_ = bboxes[j]
                    if overlaps(bbox1_xmin, bbox1_ymin, bbox1_xmax, bbox1_ymax, bbox2_xmin, bbox2_ymin, bbox2_xmax, bbox2_ymax):
                        is_occluded = is_occluded or (average_depth[j] < average_depth[i])
#                         print (f"{i} occludes {j}")
            if is_occluded == False:
                new_bboxes.append(((bbox1_xmin, bbox1_ymin), (bbox1_xmax, bbox1_ymax),bbox1_id,bbox1_label))

        for bbox_start, bbox_end, _,_ in new_bboxes:
            cv2.rectangle(img_bgr,bbox_start, bbox_end, (0,0,255))
        
        img = Image.fromarray(img_bgr[:,:,::-1])
        img.save(save_img_fpath)
        if show_depthmap:
            plt.imshow(img)        
            cm = plt.cm.get_cmap('jet')
            plt.scatter(uv1[0], uv1[1], c=1 - uv1[2] / max(uv1[2]), s=1, cmap=cm)
            plt.axis('off')
            plt.savefig("base_"+save_img_fpath)
            plt.clf()
        return pickle_format(new_bboxes,img)


In [18]:
def dump_log_2d_bboxes_to_imgs(
    log_ids: Sequence[str],
    max_num_images_to_render: int,
    data_dir: str,
    experiment_prefix: str,
    motion_compensate: bool = True,
) -> List[str]:
    """
    We bring the 3D points into each camera coordinate system, and do the clipping there in 3D.

    Args:
        log_ids: A list of log IDs
        max_num_images_to_render: maximum numbers of images to render.
        data_dir: path to dataset with the latest data
        experiment_prefix: Output directory
        motion_compensate: Whether to motion compensate when projecting

    Returns:
        saved_img_fpaths
    """
    saved_img_fpaths = []
    dl = SimpleArgoverseTrackingDataLoader(data_dir=data_dir, labels_dir=data_dir)
    avm = ArgoverseMap()

    for log_id in log_ids:
        save_dir = f"{experiment_prefix}_{log_id}"
        if not Path(save_dir).exists():
            os.makedirs(save_dir)
            
        # JUSTIN: For base image as I'm solivng sync issues
        if not Path("base_"+save_dir).exists():
            os.makedirs("base_"+save_dir)

        city_name = dl.get_city_name(log_id)
        log_calib_data = dl.get_log_calibration_data(log_id)

        flag_done = False
        for cam_idx, camera_name in enumerate(RING_CAMERA_LIST + STEREO_CAMERA_LIST):
            
            if camera_name != name_of_camera:
                continue

            print (dl.data_dir, log_id,camera_name)
            cam_im_fpaths = dl.get_ordered_log_cam_fpaths(log_id, camera_name)
            print (len(cam_im_fpaths), log_id, camera_name)
            
            
            ids = []
            for im_fpath in cam_im_fpaths:
                print (im_fpath)
                cam_timestamp = Path(im_fpath).stem.split("_")[-1]
                cam_timestamp = int(cam_timestamp)
                ply_fpath = dl.get_closest_lidar_fpath(log_id, cam_timestamp)
                if ply_fpath is None:
                    continue
                lidar_timestamp = Path(ply_fpath).stem.split("_")[-1]
                lidar_timestamp = int(lidar_timestamp)
                labels = dl.get_labels_at_lidar_timestamp(
                    log_id, lidar_timestamp)
                if labels is None:
                    logging.info("\tLabels missing at t=%s", lidar_timestamp)
                    continue
                for label in labels:
                    obj_rec = json_label_dict_to_obj_record(label)
                    track_id = (obj_rec.track_id).replace("-","")
                    ids.append(track_id)

            
            
            for i, im_fpath in enumerate(cam_im_fpaths):
                
                if i % 50 == 0:
                    logging.info("\tOn file %s of camera %s of %s", i, camera_name, log_id)

                cam_timestamp = Path(im_fpath).stem.split("_")[-1]
                cam_timestamp = int(cam_timestamp)

                # load PLY file path, e.g. 'PC_315978406032859416.ply'
                ply_fpath = dl.get_closest_lidar_fpath(log_id, cam_timestamp)
                if ply_fpath is None:
                    print ("break at no_fpath")
                    continue
                lidar_pts = load_ply(ply_fpath)# here we get lidar points
                

                
                
                save_img_fpath = f"{save_dir}/{camera_name}_{cam_timestamp}.jpg"
#                 save_img_fpath = f"{save_dir}/{camera_name}_{i}.jpg"
                print (f"Saving to {save_img_fpath}")
                if Path(save_img_fpath).exists():
                        print ("path exists exit")
                        continue

                city_to_egovehicle_se3 = dl.get_city_to_egovehicle_se3(log_id, cam_timestamp)
                if city_to_egovehicle_se3 is None:
                    print ("break at city_to_egovehicle")
                    continue

                lidar_timestamp = Path(ply_fpath).stem.split("_")[-1]
                lidar_timestamp = int(lidar_timestamp)
                labels = dl.get_labels_at_lidar_timestamp(log_id, lidar_timestamp)
                if labels is None:
                    logging.info("\tLabels missing at t=%s", lidar_timestamp)
                    print("break at missing label")
                    continue

                # Swap channel order as OpenCV expects it -- BGR not RGB
                # must make a copy to make memory contiguous
                img_bgr = imageio.imread(im_fpath)[:, :, ::-1].copy()
                camera_config = get_calibration_config(log_calib_data, camera_name)
                planes = generate_frustum_planes(camera_config.intrinsic.copy(), camera_name)

#                 print("We about to plot")
                plot_img_2d_bboxes(labels, planes, img_bgr, log_calib_data, 
                                   camera_name, cam_timestamp, lidar_timestamp, data_dir, 
                                   log_id, save_img_fpath,
                                   lidar_pts
                                  )

#                 print ("Finished plotting")
                if i > 400:
                        break

        category_subdir = "2d_amodal_labels_100fr"

        if not Path(f"{experiment_prefix}_{category_subdir}").exists():
            os.makedirs(f"{experiment_prefix}_{category_subdir}")

        for cam_idx, camera_name in enumerate(RING_CAMERA_LIST + STEREO_CAMERA_LIST):
            # Write the cuboid video -- could also write w/ fps=20,30,40
            if camera_name != name_of_camera:
                continue
            if "stereo" in camera_name:
                fps = 5
            else:
                fps = 30
            img_wildcard = f"{save_dir}/{camera_name}_%*.jpg"
            output_fpath = f"{experiment_prefix}_{category_subdir}/{log_id}_{camera_name}_{fps}fps.mp4"

            write_nonsequential_idx_video(img_wildcard, output_fpath, fps)

            print (img_wildcard, output_fpath,fps)
            print ("###################### HERE #####################")

In [19]:
# def main(args: Any):
#     """Run the example."""
#     log_ids = [log_id.strip() for log_id in args.log_ids.split(",")]
#     dump_log_2d_bboxes_to_imgs(
#         log_ids, args.max_num_images_to_render * 9, args.dataset_dir, args.experiment_prefix
#     )


# if __name__ == "__main__":
#     # Parse command line arguments
#     parser = argparse.ArgumentParser()
#     parser.add_argument(
#         "--max-num-images-to-render", default=5, type=int, help="number of images within which to render 3d cuboids"
#     )
#     parser.add_argument("--dataset-dir", type=str, required=True, help="path to the dataset folder")
#     parser.add_argument(
#         "--log-ids",
#         type=str,
#         required=True,
#         help="comma separated list of log ids, each log_id represents a log directory, e.g. found at "
#         " {args.dataset-dir}/argoverse-tracking/train/{log_id} or "
#         " {args.dataset-dir}/argoverse-tracking/sample/{log_id} or ",
#     )
#     parser.add_argument(
#         "--experiment-prefix",
#         default="output",
#         type=str,
#         help="results will be saved in a folder with this prefix for its name",
#     )
#     args = parser.parse_args()
#     logger.info(args)

#     if args.log_ids is None:
#         logger.error(f"Please provide a comma seperated list of log ids")
#         raise ValueError(f"Please provide a comma seperated list of log ids")

#     main(args)
log_ids = ['6f153f9c-edc5-389f-ac6f-40705c30d97e']
max_num_images_to_render = 1
dataset_dir = '/data/wong.justin/argo/argoverse-tracking/train1/'
experiment_prefix = 'depth'
dump_log_2d_bboxes_to_imgs(log_ids,max_num_images_to_render*9, dataset_dir, experiment_prefix)

/data/wong.justin/argo/argoverse-tracking/train1/ 6f153f9c-edc5-389f-ac6f-40705c30d97e ring_front_center
463 6f153f9c-edc5-389f-ac6f-40705c30d97e ring_front_center
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966434214396776.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966434247696632.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966434280996408.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966434314296296.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966434347596072.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/r

/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966440574695800.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966440607995688.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966440641295464.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966440674595352.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966440707895208.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966440741194984.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_3

/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966446302297512.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966446335597232.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966446368897072.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966446402196792.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966446435496512.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_315966446468796352.jpg
/data/wong.justin/argo/argoverse-tracking/train1//6f153f9c-edc5-389f-ac6f-40705c30d97e/ring_front_center/ring_front_center_3

AssertionError: 

In [4]:
import sys
sys.maxsize

9223372036854775807

In [24]:
a = np.array([0,1,2,6])
dict([(id, i)for i, id in enumerate(a)])

{0: 0, 1: 1, 2: 2, 6: 3}